In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno    # 누락값 표시
import warnings
warnings.filterwarnings('ignore')  # 워닝 무시

from sklearn.model_selection import train_test_split
%matplotlib inline

## 교차 검증(cross-validation)

#### k-fold cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

iris = load_iris()
logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data, iris.target, cv=3)
print(scores)
print(scores.mean())

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict

stkfold = StratifiedKFold(n_splits=5)
kfold = KFold(n_splits=5, shuffle=True,)
print(cross_val_score(logreg, iris.data, iris.target, cv=stkfold).mean())
cross_val_score(logreg, iris.data, iris.target, cv=kfold).mean()

#### LOOCV(LeaveOneOut)

In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print(len(scores), '{:2f}'.format(scores.mean()))

#### 임의 분할 교차 검증(ShuffleSplit)

In [ ]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(train_size=.5, test_size=.3, n_splits=10, random_state=0)
score = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
score

#### 그룹별 교차 검증(GroupKFold)

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.datasets import make_blobs

# 인위적 데이터셋 생성
X, y = make_blobs(n_samples=13, random_state=0)
groups = [0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4]
scores = cross_val_score(estimator = logreg, groups = groups, X=X, y = y, cv=GroupKFold(n_splits=3))
scores

## 그리드 서치(grid search)

#### 검증 세트 활용하기

In [ ]:
from sklearn.svm import SVC

X_trainval, X_test, y_trainval, y_test = train_test_split(iris.data, iris.target, random_state=0)
X_train, X_valid, y_train, y_valid =  train_test_split(X_trainval, y_trainval, random_state=1)
                                                      
best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_parameters = {'C':C, 'gamma':gamma}
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)

print('{:.2f}'.format(best_score))
print(best_parameters)
print('{:.2f}'.format(test_score))

#### 교차 검증 활용하기

In [ ]:
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(gamma=gamma, C=C)
        scores = cross_val_score(svm, X_trainval, y_trainval, cv=5)
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_parameters = {'C':C, 'gamma':gamma}
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)

In [ ]:
# GridSearchCV 활용하기
from sklearn.model_selection import GridSearchCV

# 딕셔너리 형태로 검색 대상 매개변수를 지정해야한다.
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
grid_search.fit(X_train, y_train)
grid_search.predict(X_test)
grid_search.score(X_test, y_test)
grid_search.best_params_

#### 교차 검증 결과 분석(시각화)

In [ ]:
import mglearn
# 검색 대상 매개변수 그리드가 2차원이므로 heatmap으로 시각화하기 좋다.

results = pd.DataFrame(grid_search.cv_results_)
scores = np.array(results.mean_test_score).reshape(6,6)

mglearn.tools.heatmap(scores, xlabel='gamma', xticklabels=param_grid['gamma'],
                     ylabel='C', yticklabels=param_grid['C'], cmap='viridis')

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(13,5))
param_grid_linear = {'C': np.linspace(1,2,6), 'gamma': np.linspace(1,2,6)}
param_grid_one_log = {'C': np.linspace(1,2,6), 'gamma': np.logspace(-3,2,6)}
param_grid_range = {'C': np.logspace(-3,2,6), 'gamma': np.logspace(-7,-2,6)}

for param_grid, ax in zip([param_grid_linear, param_grid_one_log, param_grid_range], ax):
    grid_search = GridSearchCV(SVC(), param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    scores = grid_search.cv_results_['mean_test_score'].reshape(6,6)

    scores_image = mglearn.tools.heatmap(
        scores, xlabel='gamma', ylabel='C', xticklabels=param_grid['gamma'],
        yticklabels=param_grid['C'], cmap='viridis', ax=ax)
plt.colorbar(scores_image, ax=ax.tolist())

#### 비대칭 매개변수 그리드 서치

In [ ]:
param_grid = [{'kernel': ['rbf'],'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}, 
              {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}]
grid_search = GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)
grid_search.best_params_

#### 중첩 교차 검증

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
scores = cross_val_score(grid_search, iris.data, iris.target, cv=5)
scores

In [ ]:
def nested_cv(X, y, inner_cv, outer_cv, Classifier, parameter_grid):
    outer_scores = []
    for training_samples, test_samples in outer_cv.split(X,y):
        best_params = {}
        best_score = -np.inf
        for parameters in parameter_grid:
            cv_scores = []
            
            for inner_train, inner_test in inner_cv.split(X[training_samples], y[training_samples]):
                clf = Classifier(**parameters)
                clf.fit(X[inner_train], y[inner_train])
                score = clf.score(X[inner_test], y[inner_test])
                cv_scores.append(score)
            mean_score = np.mean(cv_scores)
            if mean_score > best_score:
                best_score = mean_score  
                best_params = parameters
                
        clf = Classifier(**best_params)
        clf.fit(X[training_samples], y[training_samples])
        outer_scores.append(clf.score(X[test_samples], y[test_samples]))
    return np.array(outer_scores)

In [ ]:
from sklearn.model_selection import ParameterGrid, StratifiedKFold, StratifiedShuffleSplit
scores = nested_cv(iris.data, iris.target, StratifiedKFold(5),
                   StratifiedKFold(5), SVC, ParameterGrid(param_grid))
scores

## 평가 지표와 측정

#### 불균형 데이터셋

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
y = digits.target == 9
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=0)

from sklearn.dummy import DummyClassifier
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_most_frequent = dummy_majority.predict(X_test)
dummy = DummyClassifier(strategy='stratified').fit(X_train, y_train)
pred_stratified = dummy.predict(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = tree.predict(X_test)

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=0.1).fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)

#### 이진 분류의 평가 지표

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, pred_logreg)
print(confusion)

from sklearn.metrics import f1_score
print(f1_score(y_test, pred_logreg))

from sklearn.metrics import classification_report # 각 클래스가 양성일 때의 각 점수를 리포트한다.
print(classification_report(y_test, pred_logreg, target_names=['not 9', '9']))

In [ ]:
# 정밀도-재현율 곡선
X, y = make_blobs(n_samples=(4000,500), cluster_std=[7.0, 2], random_state=22)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

'''precision_recall_curve 함수는 가능한 모든 임계값에 대한 
   정밀도와 재현율의 값을 정렬된 리스트로 반환한다.'''
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(
    y_test, svc.decision_function(X_test))
close_default_svc = np.argmin(np.abs(thresholds))
plt.plot(precision, recall, label='svc')
plt.plot(precision[close_default_svc], recall[close_default_svc], 'o', markersize=10,
label='thr 0', fillstyle='none', c='k', mew=2)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=0, max_features=2)
rf.fit(X_train, y_train)
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(
    y_test, rf.predict_proba(X_test)[:, 1]) # predict_proba는 (n_samples, n_classes)를 반환한다.
close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
plt.plot(precision_rf, recall_rf, label='rf')
plt.plot(precision_rf[close_default_rf], recall_rf[close_default_rf], '^',
c='k', markersize=10, label='rf: thr 0.5', fillstyle='none', mew=2)

plt.xlabel('precision')
plt.ylabel('recall')
plt.legend(loc='best')

In [ ]:
# ROC
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, svc.decision_function(X_test))
close_default_svc = np.argmin(np.abs(thresholds))
plt.plot(fpr, tpr, label='ROC of SVC')
plt.plot(fpr[close_default_svc], tpr[close_default_svc], 'o', markersize=10,
label='thr 0', fillstyle='none', c='k', mew=2)

fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
close_default_rf = np.argmin(np.abs(thresholds_rf))
plt.plot(fpr_rf, tpr_rf, label='ROC of RF')
plt.plot(fpr[close_default_rf], tpr[close_default_rf], '^', markersize=10,
label='thr 0', fillstyle='none', c='k', mew=2)

plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend(loc='best')

#### 다중 분류의 평가 지표

In [ ]:
# 오차 행렬
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, random_state=0)
lr = LogisticRegression().fit(X_train, y_train)
pred = lr.predict(X_test)

sns.heatmap(confusion_matrix(y_test, pred), annot=True, cmap='RdYlGn', linewidths=0.2, 
						annot_kws={'size':10})
plt.xticks(fontsize=1)
plt.yticks(fontsize=1)
plt.xlabel('pred')
plt.ylabel('real')
plt.show

In [ ]:
# 분류 리포트
from sklearn.metrics import classification_report
classification_report(y_test, pred)

from sklearn.metrics import f1_score
f1_score(y_test, pred, average='macro')

#### 회귀의 평가 지표

#### 모델 선택에서 평가 지표 사용하기

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(SVC(), digits.data, digits.target==9, scoring='roc_auc')

from sklearn.model_selection import GridSearchCV 

param_grid = {'gamma': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(SVC(), param_grid, scoring='roc_auc').fit(X_train, y_train)